In [26]:
#=================================================#
#                  VERSION 0.02                   #
#=================================================#

#=================[DEPENDENCIES]==================#


import random #for shuffling the list of breeding paird
from IPython.display import display, clear_output #for a progress display

#===================[FUNCTIONS]===================#

#----------------[INITIALIZATION]-----------------#

def initialize(f, n, generation_size, parent_size, mutation_chance, generation_count):
    '''
    Generates and defines all necessary global variables for the algorithm, and starts the main loop
    Inputs:
        f                : (polynomial) a rational function of x with which we will define our splitting field
        n                : (integer)    the number of iterations of f(x) we will use to define our dynamical system
        generation_size  : (integer)    the number of elements in a single generation 
        parent_size      : (integer)    the number of elements from a parent generation to keep and breed
        mutation_chance  : (float)      the chance for a single coefficient to mutate when breeding two parents
        generation_count : (integer)    the total number of generations desired

    Key Variables:
        L          : the field defined by the largest linear factor of f^n(x) - x
        P          : 1-dimensional projective space over L
        D          : dynamical system over L
        comb_list  : unique pairings for pairs of numbers between 0 and parent_size
        
    '''
        
    global L, P, D, comb_list, gen_size, par_size, mut_chance
    
    L          = genSplittingField(f,n)
    P.<X,Y>    = ProjectiveSpace(L,1)
    D          = DynamicalSystem([X^2 - Y^2, Y^2])
    gen_size   = generation_size
    par_size   = parent_size
    mut_chance = mutation_chance
    comb_list  = genCombs(par_size)
    
    return generation_loop(generation_count) #this is the main loop
    
def genSplittingField(f, n):
    '''
    Generates the number field L defined over the largest linear factor of f^n(x)-x
    Inputs:
        f : (polynomial)   a rational function of x with which we will define L
        n : (integer)      number of iterations of f(x) we will use to define our dynamical system
    Return:
        L : (Number Field) the field defined by the largest linear factor of f^n(x) - x
    '''
    linear_factors = factor(nest(f,n,x) - x)
    L.<a>          = NumberField(linear_factors[len(linear_factors)-1][0])
    
    return L

def initialGen(pop_size, display = False):
    '''
    Generates the first generation for the algorithm by selecting random nonpreperiodic elements alpha from L
    Inputs:
        pop_size   : (integer) number of elements in a single generation 
        display    : (boolean) flag for dynamic progress display when choosing elements
    Return:
        generation : (list)    a list of tuples (alpha, h(alpha)), for alpha in L
    '''
    
    generation = []
    
    while len(generation) < pop_size:
        alpha = L.random_element(den_bound = 1)
        
        if not P(alpha).is_preperiodic(D):
            generation.append((alpha, D.canonical_height(P(alpha)))) #store tuple of alpha and h(alpha)
            
            if display: #dynamic display
                display_progress(len(generation), pop_size, "generating initial population")
                
    generation.sort(key = lambda x: x[1]) #sort generation in ascending order by h(alpha)
                              
    return generation

def genCombs(parent_size,k=2):
    '''
    Generates a master list of unique combinations for indices from 0 to parent_size
    Inputs:
        parent_size : (integer) number of elements from a parent generation to keep and breed
        k           : (integer) number of elements in a single combination, default 2
    Return:
        combs       : (list)    all combinations of k elements from 0 to parent_size
    '''
                              
    C     = Combinations(list(range(parent_size)),k) #list of combinations
    combs = []
                              
    for i in range(C.cardinality()): #place combinations in a list to play nice with python 
        combs.append(C.list()[i]) 
                              
    return combs

def display_progress(i, length, message):
    '''
    If called, displays the current operation count
    Variables:
        i      : (integer) current operation number
        length : (integer) total number of operations
        message: (string)  message to be displayed alongside count
    '''
                              
    clear_output(wait=True)
    display(message + "... " + str(i) +"/" + str(length))

#--------------------[SELECTION]------------------#

def selectParents(i):
    '''
    Shuffles the global list of parent combinations, then selects the first i elements
    Variables:
        i           : (integer) number of elements to select from the combination list
    Return:
        parent_list : (list)    list of i tuples
    '''
                              
    random.shuffle(comb_list)
    parent_list = comb_list[0:i]
                              
    return parent_list

def gradeGeneration(generation):
    '''
    Calculates the mean of all heights in the generation
    Inputs:
        generation : (list)  list of tuples (alpha, h(alpha))
    Return:
        fitness    : (float) mean of all heights h(alpha) from the input list
        
    '''
                              
    fitness = 0
                              
    for i in range(len(generation)):
        fitness += generation[i][1]
    fitness = fitness/len(generation)
                              
    return fitness

#--------------------[BREEDING]-------------------#
    
def breedElements(alpha,beta):
    '''
    Takes two elements of L and produces an element with coefficients randomly chosen from both
    Inputs:
        alpha : (sage number field element) an element of L
        beta  : (sage number field element) an element of L
    Return:
        gamma : (sage number field element) an element of L with coefficients from alpha, beta and mutations
    '''                          
                              
    gamma = 0
    deg   = max(alpha.polynomial().degree(), beta.polynomial().degree()) #cast alpha and beta to polynomial type for ease of use
    
    for i in range(deg + 1):
        parent_coeff = coeffProbs(mut_chance) #calculate the probability for coefficient choice based on global mutation chance
        
        if parent_coeff == 2:
            new_coeff = int(RealDistribution('gaussian', 10).get_random_element()) #if mutation triggered, use gaussian distribution to pick new coefficient
        
        elif parent_coeff == 1:
            new_coeff = alpha[i]
        
        else:
            new_coeff = beta[i]
            
        gamma += new_coeff*L.an_element()
        
    return gamma

def breedGeneration(parent_generation):
    '''
    Produces a list of elements of L bred from a list of elements of L
    Inputs:
        parent_generation : (list) a list of elements from the field L
    ReturnL
        child_gen         : (list) a list of elements bred from parent_generation, and the parents from parent_generation
    '''
    
    child_gen      = parent_generation[0:par_size]  #select the best elements from parent_generation
    breeding_pairs = selectParents(par_size) 
    
    for pair in breeding_pairs:
        alpha = parent_generation[pair[0]][0]
        beta  = parent_generation[pair[1]][0]
        gamma = breedElements(alpha,beta)
        
        while P(gamma).is_preperiodic(D): #if gamma is preperiodic, breed a new gamma
            gamma = breedElements(alpha,beta)
        
        child_gen.append([gamma, D.canonical_height(P(gamma))])
    
    child_gen.sort(key = lambda x: x[1]) #sort child_gen in ascending order by h(gamma)
    
    return child_gen

def coeffProbs(mutation_chance = mut_chance, k = 2):
    '''
    Produces an element from  probability distrubtion
    Inputs:
        mutation_chance : (float)   probability of any element mutating 
        k               : (integer) number of parents from which to draw coefficients
    Return:
        prob_dist.get_random_element() : (float) element from a probability distribution
    '''
    probs = []
    
    for i in range(k):
        probs.append((1 - mutation_chance)/k)
        
    probs.append(mutation_chance)
    prob_dist = GeneralDiscreteDistribution(probs)
    
    return prob_dist.get_random_element()
    

#--------------------[MAIN LOOP]------------------#

def generation_loop(generation_count):
    '''
    Breeds new generations until terminated
    Input:
        generation_count   : (integer) number of iterations of breeding (includes initial generation)
    return:
        current_generation : (list)    list of elements from the final generation
    '''
    generation_counter = 0
        
    while generation_counter < generation_count:
        if generation_counter == 0:
            current_generation = initialGen(gen_size, display = False)

        else:
            current_generation = breedGeneration(current_generation)

        print("Generation", str(generation_counter), "has fitness score", str(gradeGeneration(current_generation)))
        generation_counter += 1

    return current_generation
    
    
#=================================================#
#                    EXECUTION                    #
#=================================================#

#------------------[PARAMETERS]-------------------#

R.<x>            = PolynomialRing(ZZ)
phi              = x^2 - 1 
n                = 3
generation_size  = 20
parent_size      = 5
mutation_chance  = 0.1
generation_count = 20


#-------------------[EXECUTION]-------------------#

final_gen = initialize(phi, n, generation_size, parent_size, mutation_chance, generation_count)
    

Generation 0 has fitness score 2.3107896783159655689621640120
Generation 1 has fitness score 1.1596383038668119606644947583
Generation 2 has fitness score 1.2754866838475448887568366654
Generation 3 has fitness score 1.2918061525749261899260633235
Generation 4 has fitness score 0.98321523499840485395123901048
Generation 5 has fitness score 0.70141594381302146345410794087
Generation 6 has fitness score 0.74216065389228161514735995609
Generation 7 has fitness score 0.70141594381302146345410794087
Generation 8 has fitness score 0.70141594381302146345410794087
Generation 9 has fitness score 0.70141594381302146345410794087
Generation 10 has fitness score 0.70141594381302146345410794087
Generation 11 has fitness score 0.70141594381302146345410794087
Generation 12 has fitness score 0.82617231755342421959883206048
Generation 13 has fitness score 0.70141594381302146345410794087
Generation 14 has fitness score 0.70141594381302146345410794087
Generation 15 has fitness score 0.82617231755342421959